# Import and clean xAPI statements

In this notebook, we import and clean the statements collected during the evaluation of the app in a school environment. The data analysis will be performed in other notebooks, and will rely on the [**xapi_analysis**](https://stocastico.github.io/xapi_analysis/) Python package.

In [1]:
# Load aks_ai magics
%load_ext ask_ai.magics

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import json
from xapi_analysis.input_csv import *

Let's define here the file(s) we are going to use. Each trial store the statements in a specific file. The trials happened in different days, so we also want to filter the data based on the timestamp the statement was sent, to avoid loading unnecessary data (for example, statements sent while testing the system)

In [3]:
trials = { "salesianos": {"file": Path('statements_salesianos.csv'),
                        "start": "2023-03-10T10:00:00.000Z",
                        "end": "2023-03-10T18:00:00.000Z"},
         "deusto": {"file": Path('statements_deusto.csv'),
                    "start": "2023-05-09T05:00:00.000Z",
                    "end": "2023-05-09T13:00:00.000Z"},
         "xubiri": {"file": Path('statements_xubiri.csv'),
                    "start": "2023-05-10T05:00:00.000Z",
                    "end": "2023-05-10T13:00:00.000Z"}
       }

We define some simple functions for importing and cleaning the dataset

In [4]:
def read_statements(input_file: Path) -> pd.DataFrame:
    df = pd.read_csv(input_file,
                 usecols = ["timestamp", "actor", "verb", "object", "result", "stored"])

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(by=['timestamp'], ascending=True)
    return df

def filter_statements_date(df: pd.DataFrame, start_date: str, end_date: str="") -> pd.DataFrame:
    df = df[df['timestamp'] >= start_date]
    if end_date != "":
        df = df[df['timestamp'] <= end_date]
    return df
    
def clean_entry(val: str, column: str) -> str:
    tmp = json.loads(val)
    if column == 'actor':
        return tmp['name']
    elif column == 'verb':
        return tmp['display']['en-US']
    elif column == 'object':   
        return tmp['definition']['name']['en-US']
    else:
        pass # just ignore it

Now we read the statements and store them in a Pandas DataFrame, using the functions defined above.

In [5]:
statements = []

for k,v in trials.items():
    print(k)
    df = read_statements(v["file"])
    df = filter_statements_date(df, v["start"], v["end"])
    statements.append(df)
    print("***"*10)

salesianos
******************************
deusto
******************************
xubiri
******************************


Now we simplify a bit the way the information is displayed

In [6]:
for df in statements:
    for nm in ['actor', 'verb', 'object']:
        df[nm] = df[nm].map(lambda x: clean_entry(x, nm))

We should be good to go. Before saving the cleaned datasets as csv files, let's print some of the datapoints.

In [7]:
statements[0].head(10)

,timestamp,stored,actor,verb,object,result
1399,2023-03-10 11:41:29.439000+00:00,2023-03-10T11:41:29.439Z,Nuria,Logged In,Salesianos,NaN
1398,2023-03-10 11:41:41.906000+00:00,2023-03-10T11:41:41.906Z,Eider,Logged In,Salesianos,NaN
1397,2023-03-10 11:41:42.372000+00:00,2023-03-10T11:41:42.372Z,Janire,Logged In,Salesianos,NaN
1396,2023-03-10 11:42:19.063000+00:00,2023-03-10T11:42:19.063Z,Lucia,Logged In,Salesianos,NaN
1395,2023-03-10 11:42:29.061000+00:00,2023-03-10T11:42:29.061Z,unai,Logged In,Salesianos,NaN
1394,2023-03-10 11:45:09.638000+00:00,2023-03-10T11:45:09.638Z,Teacher,Logged In,Salesianos,NaN
1393,2023-03-10 11:52:00.020000+00:00,2023-03-10T11:52:00.020Z,PC006,Logged In,Salesianos,NaN
1392,2023-03-10 11:52:04.063000+00:00,2023-03-10T11:52:04.063Z,PC008,Logged In,Salesianos,NaN
1391,2023-03-10 11:52:05.177000+00:00,2023-03-10T11:52:05.177Z,Tablet1,Logged In,Salesianos,"{""score"":{""raw"":0}}"
1390,2023-03-10 11:52:05.679000+00:00,2023-03-10T11:52:05.679Z,PC004,Logged In,Salesianos,NaN


In [8]:
statements[0]['verb'].unique()

array(['Logged In', 'Placed', 'Swiped', 'Asked', 'Started', 'Logged Out',
       'Accepted', 'Set Turn', 'Suggested', 'Ran Out', 'Sent', 'Checked',
       'Assigned', 'Canceled', 'Ended'], dtype=object)

In [9]:
statements[1].head(5)

,timestamp,stored,actor,verb,object,result
1379,2023-05-09 06:14:34.396000+00:00,2023-05-09T06:14:34.396Z,Android102,Logged In,Deusto,"{""score"":{""raw"":0}}"
1378,2023-05-09 06:14:50.827000+00:00,2023-05-09T06:14:50.827Z,Android101,Logged In,Deusto,"{""score"":{""raw"":0}}"
1377,2023-05-09 06:14:58.188000+00:00,2023-05-09T06:14:58.188Z,Tablet101,Logged In,Deusto,"{""score"":{""raw"":0}}"
1376,2023-05-09 06:15:25.807000+00:00,2023-05-09T06:15:25.807Z,iPad101,Logged In,Deusto,"{""score"":{""raw"":0}}"
1375,2023-05-09 06:15:36.016000+00:00,2023-05-09T06:15:36.016Z,Android104,Logged In,Deusto,"{""score"":{""raw"":0}}"


In [10]:
statements[1]['actor'].unique()

array(['Android102', 'Android101', 'Tablet101', 'iPad101', 'Android104',
       'Tablet102', 'Teacher', 'iPhone102', 'AR4Education', 'Iphone101',
       'pc010', 'Iphone 101', 'iphone102'], dtype=object)

In [11]:
for df in statements:
    print(f'There are {len(df)} statements recorded in the trial')

There are 1400 statements recorded in the trial
There are 1380 statements recorded in the trial
There are 1197 statements recorded in the trial


In [12]:
statements[0].to_csv('statements_salesianos_clean.csv')
statements[1].to_csv('statements_deusto_clean.csv')
statements[2].to_csv('statements_xubiri_clean.csv')